In [ ]:
# Import libraries. You may or may not use all of these.
!pip install -q git+https://github.com/tensorflow/docs
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers

import tensorflow_docs as tfdocs
import tensorflow_docs.plots
import tensorflow_docs.modeling

In [ ]:
# Import data
!wget https://cdn.freecodecamp.org/project-data/health-costs/insurance.csv
dataset = pd.read_csv('insurance.csv')
dataset.tail()

In [ ]:
# Map male and female strings to 0 and 1
dataset['sex'] = dataset['sex'].map({'male': 0, 'female': 1})
print(dataset['sex'])


In [ ]:
# Assign each region in the dataset an integer
dataset['region'].value_counts()
dataset['region'] = dataset['region'].map({'northwest': 0, 'northeast': 1, 'southwest': 2, 'southeast': 3})

In [ ]:
# Assign 0 and 1 to no and yes on the smoking column
dataset['smoker'] = dataset['smoker'].map({'no': 0, 'yes': 1})

In [ ]:
# Split the dataset into a training and a test dataset
train_dataset = dataset.sample(frac=0.8)
test_dataset = dataset.drop(train_dataset.index)

In [ ]:
# Pop off the labels for both datasets
train_labels = train_dataset.pop('expenses')
test_labels= test_dataset.pop('expenses')

In [ ]:
# Print out the labels to see that it worked
print(test_labels)
print(test_dataset)

In [ ]:
# Create a normalizer layer that'll make it easier for the neural network to work with our data
normalizer = tf.keras.layers.Normalization(axis=-1)
normalizer.adapt(np.array(train_dataset))


In [ ]:
# Create the model
model = tf.keras.Sequential([
    normalizer,
    layers.Dense(64, activation='relu'),
    layers.Dense(32, activation='relu'),
    layers.Dropout(.2),
    layers.Dense(1)
])

In [ ]:
# Compile the model for training
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.1),
    loss='mean_absolute_error',
    metrics=['mean_absolute_error','mean_squared_error'])


In [ ]:
# Train the model
history = model.fit(
    train_dataset,
    train_labels,
    epochs=100,
    validation_split=0.2,
)

In [ ]:
# RUN THIS CELL TO TEST YOUR MODEL. DO NOT MODIFY CONTENTS.
# Test model by checking how well the model generalizes using the test set.
loss, mae, mse = model.evaluate(test_dataset, test_labels, verbose=2)

print("Testing set Mean Abs Error: {:5.2f} expenses".format(mae))

if mae < 3500:
  print("You passed the challenge. Great job!")
else:
  print("The Mean Abs Error must be less than 3500. Keep trying.")

# Plot predictions.
test_predictions = model.predict(test_dataset).flatten()

a = plt.axes(aspect='equal')
plt.scatter(test_labels, test_predictions)
plt.xlabel('True values (expenses)')
plt.ylabel('Predictions (expenses)')
lims = [0, 50000]
plt.xlim(lims)
plt.ylim(lims)
_ = plt.plot(lims,lims)
